In [ ]:
# %% [markdown]
# # FASE 1: Web Scraping de Reseñas
# ## Persona A - Extracción de Datos de Trustpilot

# %%
# Instalación de dependencias (solo si es necesario)
# !pip install requests beautifulsoup4 pandas

# %%
import sys
import os

# Añadir directorio de scripts al path
sys.path.append('../scripts')

# %%
# Importar el scraper
from scraper import TrustpilotScraper

# %%
# Inicializar el scraper
scraper = TrustpilotScraper()

# %%
# URL de Amazon España para análisis
amazon_url = "https://es.trustpilot.com/review/www.amazon.es"

# %%
# Ejecutar scraping (2 páginas = ~40 reseñas)
scraper.get_company_reviews(amazon_url, pages=2)

# %%
# Guardar datos crudos
df_raw = scraper.save_to_csv('../data/raw/reviews_amazon_raw.csv')

# %%
# Mostrar información del dataset
print("INFORMACIÓN DEL DATASET CRUDO")
print("-" * 40)
print(f"Total de reseñas: {len(scraper.reviews_data)}")
print(f"Columnas disponibles: {list(df_raw.columns)}")

# %%
# Visualizar las primeras reseñas
import pandas as pd
pd.set_option('display.max_colwidth', 150)

print("\nPRIMERAS 5 RESEÑAS:")
print(df_raw[['usuario', 'puntuacion', 'texto']].head())

# %%
# Análisis básico de los datos crudos
print("\nANÁLISIS BÁSICO:")
print(f"Reseñas con puntuación: {df_raw['puntuacion'].notna().sum()}")
print(f"Reseñas sin puntuación: {df_raw['puntuacion'].isna().sum()}")

# %%
# Distribución de puntuaciones
if df_raw['puntuacion'].notna().any():
    print("\nDISTRIBUCIÓN DE PUNTUACIONES:")
    print(df_raw['puntuacion'].value_counts().sort_index())
    
    # Gráfico simple de distribución
    import matplotlib.pyplot as plt
    
    plt.figure(figsize=(10, 6))
    df_raw['puntuacion'].value_counts().sort_index().plot(kind='bar', color='skyblue')
    plt.title('Distribución de Puntuaciones en Reseñas de Amazon')
    plt.xlabel('Estrellas (1-5)')
    plt.ylabel('Número de Reseñas')
    plt.xticks(rotation=0)
    plt.grid(axis='y', alpha=0.3)
    plt.tight_layout()
    plt.savefig('../visualizations/distribucion_puntuaciones_raw.png', dpi=150)
    plt.show()

# %%
# Estadísticas de longitud de texto
df_raw['longitud_texto'] = df_raw['texto'].apply(len)
print("\nESTADÍSTICAS DE LONGITUD DE TEXTO:")
print(f"Longitud máxima: {df_raw['longitud_texto'].max()} caracteres")
print(f"Longitud mínima: {df_raw['longitud_texto'].min()} caracteres")
print(f"Longitud promedio: {df_raw['longitud_texto'].mean():.0f} caracteres")

# %%
# Guardar metadatos del scraping
metadata = {
    'fecha_scraping': pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S'),
    'total_reseñas': len(df_raw),
    'empresa': 'Amazon España',
    'url_fuente': amazon_url,
    'reseñas_con_puntuacion': df_raw['puntuacion'].notna().sum(),
    'longitud_promedio_texto': round(df_raw['longitud_texto'].mean(), 2)
}

import json
with open('../data/raw/metadata_scraping.json', 'w', encoding='utf-8') as f:
    json.dump(metadata, f, indent=2, ensure_ascii=False)

print("\nMETADATOS GUARDADOS:")
for key, value in metadata.items():
    print(f"  {key}: {value}")

# %%
# Exportar dataset para la siguiente fase
print("\n" + "="*60)
print("DATASET CRUDO PREPARADO PARA LA FASE 2")
print("="*60)
print(f"Archivo guardado: ../data/raw/reviews_amazon_raw.csv")
print(f"Total registros: {len(df_raw)}")
print("\nMuestra del dataset:")
display(df_raw.head())